In [ ]:
###############################################################################
import aliases
from everest.utilities import textutils
# tuple(textutils.find_all('__getattr__', path='../everest/ptolemaic'))
# textutils.replace_all('Schematic', 'Compound', path='../everest')

In [ ]:
import abc as _abc
from collections import abc as _collabc

from everest.ptolemaic.essence import Essence as _Essence
from everest.ptolemaic.system import System as _System
from everest.ptolemaic.enumm import Enumm as _Enumm

from everest.algebra.utilities.safeiterator import \
    SafeIterator as _SafeIterator


class Expression(metaclass=_System):

    @prop
    @_abc.abstractmethod
    def atoms(self, /):
        '''Returns the atoms that ultimately make up this expression.'''
        raise NotImplementedError

    @prop
    def is_terminal(self, /) -> bool:
        '''Assesses whether the expression is made up entirely of terminals.'''
        raise NotImplementedError

    @_abc.abstractmethod
    def match(self, tokens, /):
        '''
        Returns an integer indicating how many tokens
        this expression can consume,
        or `None` if the expression fails.
        '''
        raise NotImplementedError


class Atomic(Expression):

    def atoms(self, /):
        return (self,)


class Terminal(Atomic):

    '''Represents a single terminal character.'''

    source: POS

    def match(self, tokens, /):
        try:
            token = tokens[0]
        except IndexError:
            return
        if token in self.source:
            return 1

    def is_terminal(self, /):
        return True


class NonTerminal(Atomic):

    '''Represents a single nonterminal character.'''

    symbol: POS

    # def match(self, tokens, /):
    #     if tokens[0] == self.symbol:
    #         return 1

    def is_terminal(self, /):
        return False


class Empty(Atomic):

    '''Represents the empty string.'''

    def match(self, tokens, /):
        return self(tokens)


EMPTY = Empty()


class NonAtomic(Expression):

    ...


class Unary(NonAtomic):

    subexp: POS[Expression]

    @classmethod
    def _parameterise_(cls, /, *args, **kwargs):
        params = super()._parameterise_(*args, **kwargs)
        if not isinstance(subexp := params.subexp, Expression):
            raise ValueError(subexp)
        return params

    def atoms(self, /):
        return self.subexp.atoms

    def is_terminal(self, /):
        return self.subexp.is_terminal


class OneOrMore(Unary):

    def match(self, tokens, /):
        subexp = self.subexp
        matches = subexp.match(tokens)
        if matches is None:
            return
        while True:
            sub_matches = subexp.match(tokens[matches:])
            if sub_matches is None:
                break
            matches += sub_matches
        return matches


class NotFollowedBy(Unary):

    def match(self, tokens, /):
        return self.subexp.match(tokens) is None


class Ennary(NonAtomic):

    subexps: ARGS[Expression]

    @classmethod
    def _parameterise_(cls, /, *args, **kwargs):
        params = super()._parameterise_(*args, **kwargs)
        subexps = params.subexps
        if (n := len(subexps)) < 2:
            raise ValueError(n)
        if not all(
                isinstance(failure := subexp, Expression)
                for subexp in subexps
                ):
            raise ValueError(failure)
        return params

    def atoms(self, /):
        out = []
        for subexp in self.subexps:
            for atom in subexp.atoms:
                if atom not in out:
                    out.append(atom)
        return out

    def is_terminal(self, /):
        return all(subexp.is_terminal for subexp in self.subexps)


class Seq(Ennary):

    def match(self, tokens, /):
        matches = 0
        for subexp in self.subexps:
            sub_matches = subexp.match(tokens[matches:])
            if sub_matches is None:
                return
            matches += sub_matches
        return matches


class First(Ennary):

    def match(self, tokens, /):
        for subexp in self.subexps:
            sub_matches = subexp.match(tokens)
            if sub_matches is not None:
                break
        return sub_matches


def FollowedBy(subexp, /):
    return NotFollowedBy(NotFollowedBy(subexp))


def Optional(subexp, /):
    return Seq(subexp, EMPTY)


def ZeroOrMore(subexp, /):
    return Optional(OneOrMore(subexp))


class Rule(metaclass=_System):

    symbol: POS
    expression: POS[Expression]

    @prop
    def is_terminal(self, /):
        return self.expression.is_terminal


class Leksa(metaclass=_System):

    rules: ARGS[Rule]

    @classmethod
    def _check_rule(cls, rule, /):
        if isinstance(rule, Rule):
            if rule.is_terminal:
                return True

    @classmethod
    def _parameterise_(cls, /, *args, **kwargs):
        params = super()._parameterise_(*args, **kwargs)
        if not all(cls._check_rule(failed := rule) for rule in params.rules):
            raise ValueError(failed)
        return params


# class Gramma(metaclass=_System):


#     rules: ARGS[Rule]

#     @classmethod
#     def _parameterise_(cls, /, *args, **kwargs):
#         params = super()._parameterise_(*args, **kwargs)
#         if not all(isinstance(failed := rule, Rule) for rule in params.rules):
#             raise ValueError(failed)
#         return params

#     @prop
#     def nonterminals(self, /):
#         return tuple(rule.symbol for rule in self.rules)

#     def enphrase(self, stream, /):
#         raise NotImplementedError

#     def __getitem__(self, stream, /):
#         return self.enphrase(stream)


#     class _Phrase_(mroclass, metaclass=_System):

#         gramma: POS['..']
#         symbol: POS
#         contents: POS

#         @classmethod
#         def _parameterise_(cls, /, *args, **kwargs):
#             params = super()._parameterise_(*args, **kwargs)
#             params.contents = tuple(params.contents)
#             return params

In [ ]:
exp = OneOrMore(Terminal('abc'))
display(exp)
exp.match('abada')

In [ ]:
exp = Seq(
    Terminal('abc'),
    Terminal('ABC'),
    )
exp

In [ ]:
myleksa = Leksa(Rule('foo', exp))

In [ ]:
mygramma = Gramma(
    Rule(
        'foo',
        Terminal('a'),
        ),
    )
mygramma

In [ ]:
mygramma.nonterminals

In [ ]:
if not all((failed := num) < 2 for num in (1, 2)):
    print(foo)

In [ ]:
Special.EMPTY